In [18]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import json

In [36]:
projects = ["JPetStore", "DayTrader", "AcmeAir", "Plants"]
methods = ['Bunch', 'MEM', 'FoSCI', 'Mono2Micro', 'CoGCN', 'DEEPLY', 'GDC_DVF', 'HDBSCAN', 'Mo2oM']
results = []
for proj in projects:
	for method in methods:
		with open(f"results/{method}/{method}_{proj}.json") as f:
			data = json.load(f)
		if method in ["Mo2oM", "HDBSCAN"]:
			df = pd.DataFrame(data).drop(columns=["microservices"])
		else:
			df = pd.DataFrame(data)
		if method == "Mo2oM":
			normalized_df = pd.DataFrame(MinMaxScaler().fit_transform(df.values), columns=df.columns)
			df['score'] = normalized_df["SM"] - normalized_df["ICP"] - normalized_df["IFN"] - normalized_df["NED"]
			final_df = pd.DataFrame(columns=["n_clusters", "SM", "ICP", "IFN", "NED"]).astype(dtype=df.drop(columns=["alpha", "threshold", "score"]).dtypes)
			for n in df["n_clusters"].unique():
				idx = df["score"][(df["n_clusters"] == n) & (df["threshold"] == 0.25)].argmax()
				row = df[(df["n_clusters"] == n) & (df["threshold"] == 0.25)].iloc[[idx]].drop(columns=["alpha", "score"])
				final_df = pd.concat([final_df, row])
			df = final_df.reset_index(drop=True).drop(columns=["n_clusters", "threshold"])
		elif method == "HDBSCAN":
			df = df.drop(columns=["alpha", "epsilon"])
		elif method not in ['DEEPLY', 'GDC_DVF']:
			df = df.drop(columns=["K", "BCP", "Time"])
		r = df.median().to_dict()
		r["Project"] = proj
		r["Model"] = method.replace("Mo2oM", "Mo2oM (ours)")
		results.append(r)
df = pd.DataFrame(results)
df = df[["Model", "Project", "SM", "ICP", "IFN", "NED"]]
pt = df.pivot(index="Model", columns="Project", values=["SM", "ICP", "IFN", "NED"])
pt.columns = pt.columns.swaplevel(0,1)
pt = pt.sort_index(axis=1, key=lambda x: [["JPetStore", "DayTrader", "AcmeAir", "Plants", "SM", "ICP", "IFN", "NED"].index(_) for _ in x])
print(pt.to_latex(float_format="%.3f", caption=r"Median Values of Results Across various Number of Clusters. \textbf{Bold}: best. \underline{Underline}: runner-up"))

\begin{table}
\caption{Median Values of Results Across various Number of Clusters. \textbf{Bold}: best. \underline{Underline}: runner-up}
\begin{tabular}{lrrrrrrrrrrrrrrrr}
\toprule
Project & \multicolumn{4}{r}{JPetStore} & \multicolumn{4}{r}{DayTrader} & \multicolumn{4}{r}{AcmeAir} & \multicolumn{4}{r}{Plants} \\
 & SM & ICP & IFN & NED & SM & ICP & IFN & NED & SM & ICP & IFN & NED & SM & ICP & IFN & NED \\
Model &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
Bunch & 0.111 & 0.264 & 4.250 & 0.000 & 0.121 & 0.425 & 10.400 & 0.639 & 0.044 & 0.330 & 7.333 & 0.154 & 0.096 & 0.443 & 7.667 & 0.000 \\
CoGCN & 0.079 & 0.510 & 2.531 & 0.609 & 0.086 & 0.300 & 2.600 & 0.676 & 0.038 & 0.444 & 2.800 & 0.250 & 0.083 & 0.636 & 4.938 & 0.443 \\
DEEPLY & 0.170 & 0.347 & 1.960 & 0.928 & 0.115 & 0.368 & 2.748 & 0.857 & 0.089 & 0.380 & 2.237 & 0.778 & 0.215 & 0.467 & 4.500 & 0.938 \\
FoSCI & 0.050 & 0.398 & 3.792 & 0.515 & 0.094 & 0.725 & 5.053 & 0.705 & 0.095 & 0.706 & 4.375 & 0.407 & 0.115

In [ ]:
for proj in projects:
	tex = df[df["Project"]==proj].drop(columns="Project").to_latex(float_format="%.3f", index=False, caption=proj)
	print(tex.replace(r"\caption", "\\centering\n\\caption"))
	print(r"\bigskip"*3)
	print()

\begin{table}
\centering
\caption{JPetStore}
\begin{tabular}{lrrrr}
\toprule
Model & SM & ICP & IFN & NED \\
\midrule
Bunch & 0.111 & 0.264 & 4.250 & 0.000 \\
MEM & 0.121 & 0.434 & 3.429 & 1.000 \\
FoSCI & 0.050 & 0.398 & 3.792 & 0.515 \\
Mono2Micro & 0.052 & 0.318 & 2.322 & 0.216 \\
CoGCN & 0.079 & 0.510 & 2.531 & 0.609 \\
DEEPLY & 0.170 & 0.347 & 1.960 & 0.928 \\
GDC_DVF & 0.108 & 0.274 & 1.375 & 0.351 \\
HDBSCAN & 0.035 & 0.037 & 3.000 & 1.000 \\
Mo2oM (ours) & 0.289 & 0.120 & 1.470 & 0.025 \\
\bottomrule
\end{tabular}
\end{table}

\bigskip\bigskip\bigskip

\begin{table}
\centering
\caption{DayTrader}
\begin{tabular}{lrrrr}
\toprule
Model & SM & ICP & IFN & NED \\
\midrule
Bunch & 0.121 & 0.425 & 10.400 & 0.639 \\
MEM & 0.089 & 0.355 & 3.357 & 1.000 \\
FoSCI & 0.094 & 0.725 & 5.053 & 0.705 \\
Mono2Micro & 0.084 & 0.346 & 2.421 & 0.338 \\
CoGCN & 0.086 & 0.300 & 2.600 & 0.676 \\
DEEPLY & 0.115 & 0.368 & 2.748 & 0.857 \\
GDC_DVF & 0.117 & 0.329 & 2.625 & 0.419 \\
HDBSCAN & 0.498 & 0.4

In [20]:
index = pd.MultiIndex.from_frame(df[["Project", "Model"]])
new_df = df.drop(columns=["Project", "Model"]).set_index(index)
new_tex = new_df.to_latex(float_format="%.3f", escape=False, caption="Results")
new_tex = new_tex.replace("[t]", "").replace("GDC_DVF", "GDC-DVF").replace("\\cline{1-6}", "\\midrule")
new_tex = new_tex.replace("SM", "\\textbf{SM $\\uparrow$}").replace("ICP", "\\textbf{ICP $\\downarrow$}").replace("IFN", "\\textbf{IFN $\\downarrow$}").replace("NED", "\\textbf{NED $\\downarrow$}")
new_tex = new_tex.replace("Project", "\\textbf{Dataset}").replace("Model", "\\textbf{Model}")
# new_tex = new_tex.replace("JPetStore", "\\textbf{JPetStore}").replace("DayTrader", "\\textbf{DayTrader}").replace("AcmeAir", "\\textbf{AcmeAir}").replace("Plants", "\\textbf{Plants}")
print(new_tex)

\begin{table}
\caption{Results}
\begin{tabular}{llrrrr}
\toprule
 &  & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\downarrow$} & \textbf{NED $\downarrow$} \\
\textbf{Dataset} & \textbf{Model} &  &  &  &  \\
\midrule
\multirow{9}{*}{JPetStore} & Bunch & 0.111 & 0.264 & 4.250 & 0.000 \\
 & MEM & 0.121 & 0.434 & 3.429 & 1.000 \\
 & FoSCI & 0.050 & 0.398 & 3.792 & 0.515 \\
 & Mono2Micro & 0.052 & 0.318 & 2.322 & 0.216 \\
 & CoGCN & 0.079 & 0.510 & 2.531 & 0.609 \\
 & DEEPLY & 0.170 & 0.347 & 1.960 & 0.928 \\
 & GDC-DVF & 0.108 & 0.274 & 1.375 & 0.351 \\
 & HDBSCAN & 0.035 & 0.037 & 3.000 & 1.000 \\
 & Mo2oM (ours) & 0.288 & 0.153 & 1.414 & 0.118 \\
\midrule
\multirow{9}{*}{DayTrader} & Bunch & 0.121 & 0.425 & 10.400 & 0.639 \\
 & MEM & 0.089 & 0.355 & 3.357 & 1.000 \\
 & FoSCI & 0.094 & 0.725 & 5.053 & 0.705 \\
 & Mono2Micro & 0.084 & 0.346 & 2.421 & 0.338 \\
 & CoGCN & 0.086 & 0.300 & 2.600 & 0.676 \\
 & DEEPLY & 0.115 & 0.368 & 2.748 & 0.857 \\
 & GDC-DVF & 0.117 &

In [5]:
print(r"\begin{table}")
print(r"\caption{Results}")
print(r"\centering")
print(r"\begin{tabular}{lrrrr}")
print(r"\toprule")
print("\\textbf{Model} & \\textbf{SM $\\uparrow$} & \\textbf{ICP $\\downarrow$} & \\textbf{IFN $\\downarrow$} & \\textbf{NED $\\downarrow$} \\\\")
for proj in projects:
	print(r"\midrule")
	print(r"\multicolumn{5}{c}{\textbf{"+proj+"}}\\\\")
	print(r"\midrule")
	tex = df[df["Project"]==proj].drop(columns="Project").to_latex(float_format="%.3f", index=False).replace("\\begin{tabular}{lrrrr}\n", "").replace("GDC_DVF", "GDC-DVF")
	tex = tex.replace("\\end{tabular}\n", "").replace("\\toprule\n", "").replace("\\bottomrule\n", "").replace("Model & SM & ICP & IFN & NED \\\\\n\\midrule\n", "")
	print(tex)
print(r"\bottomrule")
print(r"\end{tabular}")
print(r"\end{table}")

\begin{table}
\caption{Results}
\centering
\begin{tabular}{lrrrr}
\toprule
\textbf{Model} & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\downarrow$} & \textbf{NED $\downarrow$} \\
\midrule
\multicolumn{5}{c}{\textbf{JPetStore}}\\
\midrule
Bunch & 0.111 & 0.264 & 4.250 & 0.000 \\
MEM & 0.121 & 0.434 & 3.429 & 1.000 \\
FoSCI & 0.050 & 0.398 & 3.792 & 0.515 \\
Mono2Micro & 0.052 & 0.318 & 2.322 & 0.216 \\
CoGCN & 0.079 & 0.510 & 2.531 & 0.609 \\
DEEPLY & 0.170 & 0.347 & 1.960 & 0.928 \\
GDC-DVF & 0.108 & 0.274 & 1.375 & 0.351 \\
HDBSCAN & 0.035 & 0.037 & 3.000 & 1.000 \\
Mo2oM (ours) & 0.289 & 0.120 & 1.470 & 0.025 \\

\midrule
\multicolumn{5}{c}{\textbf{DayTrader}}\\
\midrule
Bunch & 0.121 & 0.425 & 10.400 & 0.639 \\
MEM & 0.089 & 0.355 & 3.357 & 1.000 \\
FoSCI & 0.094 & 0.725 & 5.053 & 0.705 \\
Mono2Micro & 0.084 & 0.346 & 2.421 & 0.338 \\
CoGCN & 0.086 & 0.300 & 2.600 & 0.676 \\
DEEPLY & 0.115 & 0.368 & 2.748 & 0.857 \\
GDC-DVF & 0.117 & 0.329 & 2.625 & 0.419 \\
H